In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
import os

In [2]:
qdf= pd.read_csv('../toxcast_data.csv')
smiles = qdf['smiles']

three_ds = []
failed_three_ds = []
three_ds_path = 'data/from-smiles/toxcast-3d/'
opt_three_ds = []
failed_opt_three_ds = []
opt_three_ds_path = 'data/from-smiles/toxcast-3d-opt/'
two_ds = []
two_ds_path = 'data/from-smiles/toxcast-2d/'
sdf_file = 'toxcast.sdf'
csv_file = 'toxcast.sdf.csv'

for smile in tqdm(smiles):
    try:
        mol = Chem.MolFromSmiles(smile)
        mol = Chem.AddHs(mol)
        mol.SetProp('_Name', smile)
        
        AllChem.Compute2DCoords(mol)

        # Save a copy of the 2D molecule
        two_ds.append(mol.__copy__())

        if (AllChem.EmbedMolecule(mol, randomSeed = 0xf00d, maxAttempts = 100) == -1):
            failed_three_ds.append(smile)
            continue
        # Save 3D molecule
        three_ds.append(mol.__copy__())

        if AllChem.MMFFOptimizeMolecule(mol) == -1:
            failed_opt_three_ds.append(smile)
            continue

        # Save optimized 3D molecule
        opt_three_ds.append(mol.__copy__())
    except Exception as e:
        print(e)
        continue

if not os.path.exists(three_ds_path):
    os.makedirs(three_ds_path)

if not os.path.exists(opt_three_ds_path):
    os.makedirs(opt_three_ds_path)

if not os.path.exists(two_ds_path):
    os.makedirs(two_ds_path)

with Chem.SDWriter(two_ds_path + sdf_file) as writer:
    for mol in two_ds:
        writer.write(mol)
qdf.drop(columns=['smiles'], inplace=False).to_csv(two_ds_path + csv_file, index=False)

with Chem.SDWriter(three_ds_path + sdf_file) as writer:
    for mol in three_ds:
        writer.write(mol)
qdf = qdf[~qdf['smiles'].isin(failed_three_ds)]
qdf.drop(columns=['smiles'], inplace=False).to_csv(three_ds_path + csv_file, index=False)

with Chem.SDWriter(opt_three_ds_path + sdf_file) as writer:
    for mol in opt_three_ds:
        writer.write(mol)
qdf = qdf[~qdf['smiles'].isin(failed_opt_three_ds)]
qdf.drop(columns=['smiles'], inplace=False).to_csv(opt_three_ds_path + csv_file, index=False)

  0%|          | 18/8597 [00:00<07:07, 20.09it/s][14:23:39] UFFTYPER: Unrecognized atom type: Ba (0)
[14:23:39] UFFTYPER: Unrecognized charge state for atom: 1
[14:23:39] UFFTYPER: Unrecognized atom type: Nd2+3 (1)
[14:23:39] UFFTYPER: Unrecognized charge state for atom: 1
[14:23:39] UFFTYPER: Unrecognized atom type: In2+3 (1)
[14:23:39] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[14:23:39] UFFTYPER: Unrecognized atom type: Co3+3 (0)
  1%|          | 88/8597 [00:01<02:09, 65.73it/s][14:23:40] UFFTYPER: Unrecognized atom type: Ni3+2 (0)
[14:23:40] UFFTYPER: Unrecognized charge state for atom: 1
[14:23:40] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[14:23:40] UFFTYPER: Unrecognized charge state for atom: 0
[14:23:40] UFFTYPER: Unrecognized charge state for atom: 0
[14:23:40] UFFTYPER: Unrecognized atom type: Cd+2 (0)
  2%|▏         | 208/8597 [00:05<04:36, 30.37it/s][14:23:43] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[14:23:43] UFFTYPER: Unrecognized

Python argument types in
    rdkit.Chem.rdmolops.AddHs(NoneType)
did not match C++ signature:
    AddHs(RDKit::ROMol mol, bool explicitOnly=False, bool addCoords=False, boost::python::api::object onlyOnAtoms=None, bool addResidueInfo=False)


 13%|█▎        | 1143/8597 [00:40<02:56, 42.33it/s][14:24:18] UFFTYPER: Unrecognized charge state for atom: 0
[14:24:18] UFFTYPER: Unrecognized atom type: Gd2+3 (0)
 14%|█▍        | 1203/8597 [00:41<02:24, 51.23it/s][14:24:20] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[14:24:20] UFFTYPER: Unrecognized charge state for atom: 1
[14:24:20] UFFTYPER: Unrecognized atom type: Mn2+2 (0)
[14:24:20] UFFTYPER: Unrecognized charge state for atom: 7
 15%|█▌        | 1298/8597 [00:44<03:48, 31.88it/s][14:24:22] UFFTYPER: Unrecognized charge state for atom: 0
[14:24:22] UFFTYPER: Unrecognized atom type: Gd2+3 (0)
 17%|█▋        | 1455/8597 [00:53<08:35, 13.86it/s][14:24:31] UFFTYPER: Unrecognized atom type: Ti1+4 (5)
[14:24:31] UFFTYPER: Unrecognized atom type: Cr1+3 (0)
[14:24:31] UFFTYPER: Unrecognized atom type: Ni3+2 (0)
 17%|█▋        | 1481/8597 [00:55<07:38, 15.51it/s][14:24:33] UFFTYPER: Unrecognized atom type: Co3+3 (0)
[14:24:33] UFFTYPER: Unrecognized charge state for atom: 7

Python argument types in
    rdkit.Chem.rdmolops.AddHs(NoneType)
did not match C++ signature:
    AddHs(RDKit::ROMol mol, bool explicitOnly=False, bool addCoords=False, boost::python::api::object onlyOnAtoms=None, bool addResidueInfo=False)


 21%|██▏       | 1831/8597 [01:14<10:10, 11.08it/s][14:24:52] UFFTYPER: Warning: hybridization set to SP3 for atom 7
[14:24:52] UFFTYPER: Unrecognized hybridization for atom: 2
[14:24:52] UFFTYPER: Unrecognized atom type: Au+3 (2)
 22%|██▏       | 1886/8597 [01:18<04:14, 26.41it/s]

...

 94%|█████████▎| 8046/8597 [05:56<00:29, 18.98it/s][14:29:34] SMILES Parse Error: syntax error while parsing: FAIL
[14:29:34] SMILES Parse Error: Failed parsing SMILES 'FAIL' for input: 'FAIL'
 94%|█████████▎| 8052/8597 [05:56<00:23, 23.54it/s]

Python argument types in
    rdkit.Chem.rdmolops.AddHs(NoneType)
did not match C++ signature:
    AddHs(RDKit::ROMol mol, bool explicitOnly=False, bool addCoords=False, boost::python::api::object onlyOnAtoms=None, bool addResidueInfo=False)


 95%|█████████▌| 8182/8597 [05:59<00:06, 60.55it/s][14:29:37] SMILES Parse Error: syntax error while parsing: FAIL
[14:29:37] SMILES Parse Error: Failed parsing SMILES 'FAIL' for input: 'FAIL'
[14:29:37] UFFTYPER: Unrecognized charge state for atom: 17
 95%|█████████▌| 8189/8597 [05:59<00:07, 52.96it/s]

Python argument types in
    rdkit.Chem.rdmolops.AddHs(NoneType)
did not match C++ signature:
    AddHs(RDKit::ROMol mol, bool explicitOnly=False, bool addCoords=False, boost::python::api::object onlyOnAtoms=None, bool addResidueInfo=False)


 96%|█████████▋| 8276/8597 [06:00<00:05, 58.52it/s][14:29:39] UFFTYPER: Unrecognized hybridization for atom: 1
[14:29:39] UFFTYPER: Unrecognized atom type: Pt+2 (1)
100%|█████████▉| 8588/8597 [06:10<00:00, 13.30it/s][14:29:48] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[14:29:48] UFFTYPER: Unrecognized charge state for atom: 1
[14:29:48] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[14:29:48] UFFTYPER: Unrecognized charge state for atom: 1
100%|█████████▉| 8592/8597 [06:10<00:00, 12.97it/s][14:29:48] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[14:29:48] UFFTYPER: Unrecognized charge state for atom: 1
[14:29:48] UFFTYPER: Unrecognized charge state for atom: 14
100%|██████████| 8597/8597 [06:10<00:00, 23.18it/s]


In [5]:
sdf = Chem.SDMolSupplier('data/from-smiles/gdb8-3d/gdb8.sdf')

In [22]:
try:
    mol = Chem.MolFromSmiles('[H]C([H])([H])C12C([H])([H])N(C1([H])[H])C2([H])[H]')
    mol = Chem.AddHs(mol)
    print(AllChem.EmbedMolecule(mol))
    print(AllChem.EmbedMolecule(mol))
except:
    print('Failed to generate conformer for unrecognized molecules')

-1
-1
